In [8]:
import datetime
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [9]:
conn_str = """
    Driver={ODBC Driver 17 for SQL Server};
    Server=109.71.11.235;
    Database=GFC.Hackathon;
    UID=participant;
    PWD=AiNRYmEatiBL;
"""

conn = pyodbc.connect(conn_str)

In [10]:
# пример запросов - здесь выведен весь датасет признаков
query_features = "select * from dbo.Features"
df = pd.read_sql(query_features, conn)
df

,LocationId,ProductId,ValidationDateTime,Probability,IsCorrect,Column6,Column7,Column8,Column9,Column10,...,Column58,Column59,Column60,Column61,Column62,Column63,Column64,Column65,Column66,Column67
0,63,288527,2023-06-28 14:57:07,0.416474,False,86.0,85.0,0.011628,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
1,63,288606,2023-06-01 15:32:31,0.687710,False,8.0,7.0,0.125000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
2,63,288606,2023-06-03 12:55:49,0.810382,False,10.0,9.0,0.100000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
3,63,288606,2023-06-07 14:57:38,0.312500,False,4.0,3.0,0.250000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
4,63,288606,2023-06-09 14:56:35,0.605429,False,6.0,5.0,0.166667,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176613,2950,576631,2023-07-12 15:17:30,0.542776,True,1.0,0.0,1.000000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
176614,2950,576631,2023-07-15 15:02:34,0.497054,True,4.0,4.0,0.000000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
176615,2950,576631,2023-07-26 15:01:27,0.930926,None,1.0,0.0,1.000000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
176616,2950,576631,2023-07-28 15:04:19,0.704421,None,3.0,2.0,0.333333,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0


In [11]:
df['ValidationDateTime'] = pd.to_datetime(df['ValidationDateTime'])

In [24]:
# для обучения модели используем записи с IsCorrect is not null
train_query = "select * from dbo.Features f where f.IsCorrect is not null"
train_df = pd.read_sql(train_query, conn)
train_df

,LocationId,ProductId,ValidationDateTime,Probability,IsCorrect,Column6,Column7,Column8,Column9,Column10,...,Column58,Column59,Column60,Column61,Column62,Column63,Column64,Column65,Column66,Column67
0,63,288527,2023-06-28 14:57:07,0.416474,False,86.0,85.0,0.011628,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
1,63,288606,2023-06-01 15:32:31,0.687710,False,8.0,7.0,0.125000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
2,63,288606,2023-06-03 12:55:49,0.810382,False,10.0,9.0,0.100000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
3,63,288606,2023-06-07 14:57:38,0.312500,False,4.0,3.0,0.250000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
4,63,288606,2023-06-09 14:56:35,0.605429,False,6.0,5.0,0.166667,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135842,2950,574768,2023-07-19 15:01:04,0.656481,True,0.0,-1.0,NaN,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
135843,2950,575450,2023-06-20 15:00:06,0.778028,False,1.0,0.0,1.000000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
135844,2950,576459,2023-07-02 15:01:13,0.248685,False,41.0,40.0,0.024390,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0
135845,2950,576631,2023-07-12 15:17:30,0.542776,True,1.0,0.0,1.000000,None,None,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,None,0.0


In [25]:
# тестовая выборка (то, что используется в финальном зачете) - записи с IsCorrect is null
test_query = "select * from dbo.Features f where f.IsCorrect is null"
test_df = pd.read_sql(test_query, conn)
test_df

,LocationId,ProductId,ValidationDateTime,Probability,IsCorrect,Column6,Column7,Column8,Column9,Column10,...,Column58,Column59,Column60,Column61,Column62,Column63,Column64,Column65,Column66,Column67
0,63,288606,2023-07-25 14:57:04,0.406708,None,14.0,13.0,0.071429,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
1,63,288606,2023-08-03 14:58:53,0.460643,None,2.0,1.0,0.500000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
2,63,288783,2023-07-24 14:57:11,0.404959,None,4.0,4.0,0.000000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
3,63,288783,2023-08-03 14:57:39,0.659497,None,0.0,-1.0,NaN,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
4,63,289136,2023-07-24 14:57:09,0.299769,None,6.0,6.0,0.000000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40766,2950,572875,2023-07-26 15:01:26,0.248685,None,19.0,18.0,0.052632,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
40767,2950,574768,2023-07-31 15:07:00,0.173554,None,10.0,9.0,0.100000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
40768,2950,576631,2023-07-26 15:01:27,0.930926,None,1.0,0.0,1.000000,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0
40769,2950,576631,2023-07-28 15:04:19,0.704421,None,3.0,2.0,0.333333,None,None,...,0.0,0.0,0.0,0.0,1,0,0.0,0,None,0


In [26]:
train_df.sort_values('ValidationDateTime', inplace=True)
test_df.sort_values('ValidationDateTime', inplace=True)

In [15]:
# дополнительные таблицы с почасовыми продажами и остатками - мы ждем отвас новых интересных признаков :)
sales_query = "select * from dbo.LocationStateHourSales" 
sales = pd.read_sql(sales_query, conn).rename({'Datetime': 'timestamp'}, axis=1)
sales

,LocationId,ProductId,Datetime,Quantity,PriceSum,Date
0,63,288606,2023-05-01 12:00:00,6.0,179.940002,2023-05-01
1,63,288606,2023-05-01 13:00:00,2.0,59.980000,2023-05-01
2,63,288783,2023-05-01 17:00:00,2.0,259.799988,2023-05-01
3,63,288913,2023-05-01 16:00:00,1.0,129.899994,2023-05-01
4,63,289117,2023-05-01 11:00:00,3.0,92.970001,2023-05-01
...,...,...,...,...,...,...
81932787,2950,576956,2023-08-04 08:00:00,1.0,34.990002,2023-08-04
81932788,2950,576956,2023-08-04 13:00:00,1.0,34.990002,2023-08-04
81932789,2950,576967,2023-08-04 18:00:00,3.0,89.970001,2023-08-04
81932790,2950,577090,2023-08-04 22:00:00,1.0,59.990002,2023-08-04


In [16]:
stocks_query = "select * from dbo.LocationStateStocks" 
stocks = pd.read_sql(stocks_query, conn).rename({'Datetime': 'timestamp'}, axis=1)
stocks

,LocationId,ProductId,Datetime,Quantity,Date
0,63,288524,2023-05-01 00:00:00.000,21.0,2023-05-01
1,63,288527,2023-05-01 00:00:00.000,181.0,2023-05-01
2,63,288606,2023-05-01 00:00:00.000,6.0,2023-05-01
3,63,288783,2023-05-01 00:00:00.000,11.0,2023-05-01
4,63,288817,2023-05-01 00:00:00.000,18.0,2023-05-01
...,...,...,...,...,...
113395989,2950,577062,2023-08-04 00:51:28.860,10.0,2023-08-04
113395990,2950,577063,2023-08-04 00:51:28.860,18.0,2023-08-04
113395991,2950,577090,2023-08-04 00:51:28.860,8.0,2023-08-04
113395992,2950,577093,2023-08-04 00:51:28.860,21.0,2023-08-04


In [ ]:
test_df.to_parquet('test_df.parquet')
train_df.to_parquet('train_df.parquet')

sales.to_parquet('sales.parquet')
stocks.to_parquet('stocks.parquet')

In [ ]:
train_df = pd.read_parquet('train_df.parquet')
test_df = pd.read_parquet('test_df.parquet')

sales = pd.read_parquet('sales.parquet')
stocks = pd.read_parquet('stocks.parquet')

In [27]:
target = 'IsCorrect'

stocks['Date'] = stocks['Date'].apply(str)
sales['Date'] = sales['Date'].apply(str)
train_df['Date'] = train_df['ValidationDateTime'].dt.strftime('%Y-%m-%d')
test_df['Date'] = test_df['ValidationDateTime'].dt.strftime('%Y-%m-%d')

all_df = pd.concat([train_df, test_df], axis=0)
all_df.to_parquet('all_df.parquet')

m1 = all_df.merge(stocks, how='inner', on=['LocationId', 'ProductId', 'Date'], suffixes=('_train', '_stocks'))
to_drop = m1[(m1['timestamp'] > m1['ValidationDateTime'])][['LocationId', 'ProductId', 'ValidationDateTime']]
to_drop.to_parquet('drop.parquet')



In [30]:
train_df['Date'] = train_df['ValidationDateTime'].dt.strftime('%Y-%m-%d')
test_df['Date'] = test_df['ValidationDateTime'].dt.strftime('%Y-%m-%d')

new_sales = sales.groupby(['LocationId', 'ProductId', 'Date']).agg({'PriceSum': 'sum', 'Quantity': 'sum', 'timestamp': 'last'}).reset_index()
merged2 = stocks.merge(new_sales, how='left', on=['LocationId', 'ProductId', 'Date'], suffixes=('_stocks', '_sales'))
merged2.to_parquet('merged2.parquet')

In [32]:
import polars as pl

pdf = pl.read_parquet('merged2.parquet').drop(['timestamp_stocks', 'timestamp_sales']).drop('__index_level_0__')
all_df = pl.read_parquet('all_df.parquet').drop('__index_level_0__')
to_drop = pl.read_parquet('drop.parquet').drop('__index_level_0__')

In [33]:
pdf = pdf.with_columns(
    pl.col("ProductId").cast(pl.Int32).alias("ProductId"),
    pl.col("LocationId").cast(pl.Int32).alias("LocationId"),
    pl.col("Quantity_stocks").cast(pl.Float32).alias("Quantity_stocks"),
    pl.col("Quantity_sales").cast(pl.Float32).alias("Quantity_sales"),
    pl.col("PriceSum").cast(pl.Float32).alias("PriceSum"),
)

all_df = all_df.with_columns(
    pl.col("ProductId").cast(pl.Int32).alias("ProductId"),
    pl.col("LocationId").cast(pl.Int32).alias("LocationId"),
)

to_drop = to_drop.with_columns(
    pl.col("ProductId").cast(pl.Int32).alias("ProductId"),
    pl.col("LocationId").cast(pl.Int32).alias("LocationId"),
)

In [34]:
all_df = all_df.with_columns(pl.col("ValidationDateTime").dt.strftime('%Y-%m-%d').alias("Date"))
pdf = pdf.join(all_df[['LocationId', 'ProductId', 'Date', 'ValidationDateTime']], on=['LocationId', 'ProductId', 'Date'], how='left')
pdf = pdf.join(to_drop, on=['LocationId', 'ProductId', 'ValidationDateTime'], how='anti').drop('ValidationDateTime').unique()

pdf = pdf.with_columns(pl.lit((pdf['LocationId'] + '_' + pdf['ProductId'])).alias("LocationId_ProductId"), pl.col("Date").str.to_datetime('%Y-%m-%d'))
pdf = pdf.rename({'Date': 'timestamp'})

In [35]:
pdf = (pdf.with_columns(
    pl.when(pl.col("Quantity_stocks") < 0)
      .then(0)
      .otherwise(pl.col("Quantity_stocks")).alias("Quantity_stocks"),
    pl.when(pl.col("Quantity_sales") < 0)
      .then(0)
      .otherwise(pl.col("Quantity_sales")).alias("Quantity_sales"),
))

pdf = pdf.sort(['LocationId_ProductId', 'timestamp', ])

pdf_concat = [pdf
    .rolling("timestamp", period=f"{i}d", by=["LocationId_ProductId"], check_sorted=False)
    .agg(
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).mean().prefix(f"mean_{i}_"),
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).max().prefix(f"max_{i}_"),
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).min().prefix(f"min_{i}_"),
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).std().prefix(f"std_{i}_"),
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).sum().prefix(f"sum_{i}_"),
    ).drop(['timestamp', 'LocationId_ProductId']) for i in list(range(3, 8)) + [180]]

gloc_join = [pdf.sort(['LocationId', 'timestamp', ])
 .rolling("timestamp", period=f"{i}d", by=["LocationId"], check_sorted=False)
    .agg(
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).mean().prefix(f"mean_gloc{i}_"),
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).max().prefix(f"max_gloc{i}_"),
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).min().prefix(f"min_gloc{i}_"),
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).std().prefix(f"std_gloc{i}_"),
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).sum().prefix(f"sum_gloc{i}_"),
    ) for i in list(range(3, 8)) + [180]]

gprod_join = [pdf.sort(['ProductId', 'timestamp', ])
 .rolling("timestamp", period=f"{i}d", by=["ProductId"], check_sorted=False)
    .agg(
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).mean().prefix(f"mean_gprod{i}_"),
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).max().prefix(f"max_gprod{i}_"),
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).min().prefix(f"min_gprod{i}_"),
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).std().prefix(f"std_gprod{i}_"),
        pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).sum().prefix(f"sum_gprod{i}_"),
    ) for i in list(range(3, 8)) + [180]]

loc_join = pdf.group_by(['LocationId', 'timestamp']).agg(
            pl.col(['Quantity_stocks',]).mean().prefix(f"mean_loc_"),
            pl.col(['Quantity_stocks',]).max().prefix(f"max_loc_"),
            pl.col(['Quantity_stocks', ]).min().prefix(f"min_loc_"),
            pl.col(['Quantity_stocks', ]).std().prefix(f"std_loc_"),
            pl.col(['Quantity_stocks', ]).sum().prefix(f"sum_loc_"),
            pl.col(['Quantity_stocks',]).drop_nulls().count().prefix(f"count_loc_"),
        ),
prod_join =  pdf.group_by(['ProductId', 'timestamp']).agg(
            pl.col(['Quantity_stocks', ]).mean().prefix(f"mean_prod_"),
            pl.col(['Quantity_stocks', ]).max().prefix(f"max_prod_"),
            pl.col(['Quantity_stocks', ]).min().prefix(f"min_prod_"),
            pl.col(['Quantity_stocks', ]).std().prefix(f"std_prod_"),
            pl.col(['Quantity_stocks', ]).sum().prefix(f"sum_prod_"),
            pl.col(['Quantity_stocks',]).drop_nulls().count().prefix(f"count_prod_")
        ),
# можем так как проверка стоков проводится до validationdatetime (дропнуто, если это не так)

In [36]:
all_gloc_join = pl.concat(gloc_join[:1] + [x.drop(['LocationId', 'timestamp']) for x in gloc_join[1:]], how="horizontal",)
all_gprod_join = pl.concat(gprod_join[:1] + [x.drop(['ProductId', 'timestamp']) for x in gprod_join[1:]], how="horizontal",)

pdf = pl.concat([pdf.sort(['LocationId', 'timestamp']), all_gloc_join.drop(['LocationId', 'timestamp'])], how="horizontal")
pdf = pl.concat([pdf.sort(['ProductId', 'timestamp']), all_gprod_join.drop(['ProductId', 'timestamp'])], how="horizontal")

pdf = pdf.join(loc_join[0], on = ['LocationId', 'timestamp'], how='left')
pdf = pdf.join(prod_join[0], on = ['ProductId', 'timestamp'], how='left')

pdf = pdf.sort(['LocationId_ProductId', 'timestamp', ])
pdf =pl.concat([pdf] + pdf_concat, how="horizontal",)

pdf = pdf.with_columns(
    *[pl.col(['Quantity_stocks', 'Quantity_sales', 'PriceSum']).shift(i).over(['LocationId_ProductId']).prefix(f"prev_{i}_") for i in range(1, 8)],
    *[pl.col(['Quantity_stocks',]).diff(i).over(['LocationId_ProductId']).prefix(f"diff_{i}") for i in range(1, 4)] ,
)

pdf = pdf.drop(['Quantity_sales', 'PriceSum'])

In [37]:
train_df = pl.read_parquet('train_df.parquet')
test_df = pl.read_parquet('test_df.parquet')

train_df = train_df.with_columns(
    pl.col("ProductId").cast(pl.Int32).alias("ProductId"),
    pl.col("LocationId").cast(pl.Int32).alias("LocationId"),
)
test_df = test_df.with_columns(
    pl.col("ProductId").cast(pl.Int32).alias("ProductId"),
    pl.col("LocationId").cast(pl.Int32).alias("LocationId"),
)

train_df = train_df.with_columns(
    pl.col('ValidationDateTime').dt.strftime('%Y-%m-%d').alias('Date'),
)

test_df = test_df.with_columns(
    pl.col('ValidationDateTime').dt.strftime('%Y-%m-%d').alias('Date'),
)

pdf = pdf.with_columns(
    pl.col('timestamp').dt.strftime('%Y-%m-%d').alias('Date'),
)

train_df = train_df.join(pdf, on=['LocationId', 'ProductId', 'Date'], how='left')
test_df = test_df.join(pdf, on=['LocationId', 'ProductId', 'Date'], how='left')

train_df.write_parquet('train_with_features_final.parquet')
test_df.write_parquet('test_with_features_final.parquet')